In [2]:
import pandas as pd 

path = "11.deep/d0726/"
path = ""
chatbotData=pd.read_csv(path+"ChatBotData.csv")
question, answer = list(chatbotData["Q"]) , list(chatbotData["A"])
print(len(question))
print(len(answer))

11823
11823


In [3]:
import re 
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import os 
from konlpy.tag import Okt 
from keras import models, layers, optimizers, metrics, preprocessing 

# 태그단어 
PAD = "<PADDING>"  # 패딩 
STA = "<START>"    # 시작 
END = "<END>"      # 끝 
OOV = "<OOV>"      # out of vocabulary 
PAD_INDEX = 0
STA_INDEX = 1 
END_INDEX = 2 
OOV_INDEX = 3 

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수 
maxSequences = 30 

# 임베딩 벡터 차원 
embeddingDim = 100 

# LSTM 출력 차원 
lstmHiddenDim = 128 

# 정규표현식 필터 
RE_FILTER = re.compile("[.,!?\':;~()]'")

In [4]:
# 형태소 분석기 
def posTag(sentences): 
    tagger = Okt()
    sentencePos = []
    for sentence in sentences:
        # 특수문자 제거 
        sentence = re.sub( RE_FILTER  ,  ""  , sentence )
        sentence = " ".join(tagger.morphs(sentence))
        sentencePos.append(sentence)
    return sentencePos 
    
question = posTag(question)
answer = posTag(answer)

# 질문 + 대답 을 하나로 합치기 
sentences = [] 
sentences.extend(question)
sentences.extend(answer)
len(sentences)

23646

In [9]:
words = []
# 단어배열 생성 
for sentence in sentences : 
    for word in sentence.split():
        words.append(word)


# words에서 길이가 0인 단어를 삭제 
# 중복단어를 삭제 
words = [ word for word in words if len(word)>0 ] # 길이가 0인 단어 삭제 
words = list(set(words)) # 중복 단어 삭제 

words[:0] = [PAD,STA,END,OOV]

# 단어에 대한 인덱스를 부여 -> 딕셔너리
wordToIndex = {word:index for index, word in enumerate(words)}
indexToWord = {index:word for index, word in enumerate(words)}
indexToWord[5]

'바람난'

In [10]:
# 문장 -> 인덱스로 변환
def convertTextToIndex(sentences, voc, mytype):
    
    sentencesIndex=[]
    for sentence in sentences:
        
        sentenceIndex=[]
        if mytype == DECODER_INPUT:
            
            sentenceIndex.extend([voc[STA]])
            
        for word in sentence.split():
            
            if voc.get(word) is not None: # 단어에 해당하는 인덱스가 있는 경우
                sentenceIndex.extend([voc[word]]) # 단어에 해당되는 인덱스가 추가
                
            else: # 사전에 없는 단어의 경우 OOV추가
                sentenceIndex.extend([voc[OOV]])
                
        if mytype == DECODER_TARGET:
            
            # 디코더 출력은 맨 마지막에 end 추가
            if maxSequences <= len(sentenceIndex):
                sentenceIndex = sentenceIndex[:maxSequences-1] + [voc[END]]
                
            else:
                sentenceIndex += [voc[END]]
                
        else:
            if len(sentenceIndex) > maxSequences:
                sentenceIndex = sentenceIndex[:maxSequences]
                
        # 0으로 채움(pad_sequence)
        sentenceIndex += [wordToIndex[PAD]] * (maxSequences-len(sentenceIndex))
        
        sentencesIndex.append(sentenceIndex)
            
    return np.asarray(sentencesIndex)

In [11]:
# 인코더 입력, 디코더 입력, 디코더 출력 -> 인덱스 변환
xEncoder = convertTextToIndex(question, wordToIndex, ENCODER_INPUT)
print(xEncoder[0])

xDecoder = convertTextToIndex(answer, wordToIndex, DECODER_INPUT)
print(xDecoder[0])

yDecoder = convertTextToIndex(answer, wordToIndex, DECODER_TARGET)
print(yDecoder[0])

[ 658 5466 1517    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[    1   777  8433  2604 11914  1193     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
[  777  8433  2604 11914  1193     2     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]


In [12]:
#np.zeros((2,3,4))
oneHotData=np.zeros((len(yDecoder),maxSequences,len(words)))
#100(답변개수), 30(최대 단어 개수), 454(전체 단어 집합 개수)

np.shape(oneHotData)#디코더 출력
#하루 가 또 가네요 END
for i, seq in enumerate(yDecoder):# (100,30)
    for j, index in enumerate(seq):
        oneHotData[i,j,index]=1
yDecoder=oneHotData

In [13]:
#훈련 모델 생성
#인코더 정의

#입력 문장의 인덱스 sequence를 입력
encoderInputs=layers.Input(shape=(None,))
#임베딩 계층
encoderOutputs=layers.Embedding(len(words),embeddingDim)(encoderInputs)

encoderOutputs,stateH, stateC=layers.LSTM(lstmHiddenDim,return_state=True, 
            dropout=0.2, recurrent_dropout=0.5)(encoderOutputs)
#return_state=True => 상태값 리턴
#LSTM은 2개 상태 존재(셀, 히든 스테이트)

encoderStates=[stateH, stateC]


#디코더 정의
#출력 문장의 인덱스 sequence를 입력
decoderInputs=layers.Input(shape=(None,))
#임베딩 계층
decoderEmbedding=layers.Embedding(len(words),
                                embeddingDim)
decoderOutputs=decoderEmbedding(decoderInputs)



decoderLSTM=layers.LSTM(lstmHiddenDim,
                        return_state=True, 
            return_sequences=True, 
                        dropout=0.2, 
                        recurrent_dropout=0.5)
decoderOutputs, _, _=decoderLSTM(decoderOutputs,initial_state=encoderStates)
decoderDense=layers.Dense(len(words), 
                          activation="softmax")
decoderOutputs=decoderDense(decoderOutputs)

In [14]:
model=models.Model([encoderInputs, decoderInputs],
             decoderOutputs)
             
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
             
#예측 모델 인코더 정의
encoderModel=models.Model(encoderInputs, 
                          encoderStates)

#예측 모델 디코더 정의
#바로 앞에 있는 디코더의 출력(상태)을 입력 받아서
#예측을 해야 함.
decoderStateInputH=layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC=layers.Input(shape=(lstmHiddenDim,))
decoderStatesInputs=[decoderStateInputH,decoderStateInputC]

#임베딩 계층
decoderOutputs=decoderEmbedding(decoderInputs)
#LSTM 계층
decoderOutputs, stateH, stateC=decoderLSTM(decoderOutputs,
           initial_state=decoderStatesInputs)
decoderStates=[stateH, stateC]

#Dense계층을 통해 원핫 형식으로 예측 단어 인덱스를 추출
decoderOutputs=decoderDense(decoderOutputs)

#예측 모델 디코더 설정
decoderModel=models.Model([decoderInputs]+decoderStatesInputs,
            [decoderOutputs]+decoderStates)